In [1]:
import pandas as pd
import preprocessor as p
import string
import re
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from wordcloud import WordCloud
import warnings
warnings.filterwarnings("ignore")
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

pd.set_option('display.max_columns', None)
df = pd.read_csv("Datasets/Agaton17k.csv")

df.head()

,Date,Tweet Id,Tweet Content,Tweet URL,Username,Tweet Language,Tweet Media,Tweet Coordinates,Tweet Place,Hastags,Cash Tags
0,2022-04-21 22:13:09+00:00,1517265208657727488,"#SonshineNewsblast: @doe_ph, nagdeklara ng pri...",https://twitter.com/DZAR1026/status/1517265208...,DZAR1026,tl,NaN,NaN,NaN,"['SonshineNewsblast', 'AgatonPH']",NaN
1,2022-04-21 21:10:35+00:00,1517249460857835520,Death toll sa Bagyong #AgatonPH lumobo na sa 2...,https://twitter.com/dzrhnews/status/1517249460...,dzrhnews,tl,NaN,NaN,NaN,"['AgatonPH', 'MagandangUmagaPilipinas', 'SamaS...",NaN
2,2022-04-21 20:01:13+00:00,1517232006299631617,#ICYMI: Angel Locsin explained that she chose ...,https://twitter.com/inquirerdotnet/status/1517...,inquirerdotnet,en,NaN,NaN,NaN,"['ICYMI', 'AgatonPH']",NaN
3,2022-04-21 19:15:00+00:00,1517220373821067269,BINAWI ng National Disaster Risk Reduction and...,https://twitter.com/AbanteNews/status/15172203...,AbanteNews,tl,NaN,NaN,NaN,"['TunayNaTabloidista', 'AbanteNews', 'AgatonPH']",NaN
4,2022-04-21 16:48:11+00:00,1517183425853083666,تحديث | أفاد المجلس الوطني للحد من مخاطر الكوا...,https://twitter.com/Philippine_live/status/151...,Philippine_live,ar,[Photo(previewUrl='https://pbs.twimg.com/media...,NaN,NaN,['AgatonPH'],NaN


In [2]:
temp = df[(df["Tweet Language"] == "tl")]
temp.head()

,Date,Tweet Id,Tweet Content,Tweet URL,Username,Tweet Language,Tweet Media,Tweet Coordinates,Tweet Place,Hastags,Cash Tags
0,2022-04-21 22:13:09+00:00,1517265208657727488,"#SonshineNewsblast: @doe_ph, nagdeklara ng pri...",https://twitter.com/DZAR1026/status/1517265208...,DZAR1026,tl,NaN,NaN,NaN,"['SonshineNewsblast', 'AgatonPH']",NaN
1,2022-04-21 21:10:35+00:00,1517249460857835520,Death toll sa Bagyong #AgatonPH lumobo na sa 2...,https://twitter.com/dzrhnews/status/1517249460...,dzrhnews,tl,NaN,NaN,NaN,"['AgatonPH', 'MagandangUmagaPilipinas', 'SamaS...",NaN
3,2022-04-21 19:15:00+00:00,1517220373821067269,BINAWI ng National Disaster Risk Reduction and...,https://twitter.com/AbanteNews/status/15172203...,AbanteNews,tl,NaN,NaN,NaN,"['TunayNaTabloidista', 'AbanteNews', 'AgatonPH']",NaN
6,2022-04-21 16:40:00+00:00,1517181366160412675,#FrontlineTonight | Plano ng COMELEC na magpat...,https://twitter.com/News5PH/status/15171813661...,News5PH,tl,[Video(thumbnailUrl='https://pbs.twimg.com/amp...,NaN,NaN,"['FrontlineTonight', 'AgatonPH', 'BilangPilipi...",NaN
11,2022-04-21 13:40:23+00:00,1517136166923173894,Brgy.Pilar\nAbuyog Leyte\n#AgatonPH \n#abuyogn...,https://twitter.com/nayer_143/status/151713616...,nayer_143,tl,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,NaN,NaN,"['AgatonPH', 'abuyogneedshelp']",NaN


In [3]:
def preprocessing_text(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+https\S+", '',text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#','',text)
    text = re.sub(r'[^\w\s]','',text)
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

tweet_df = pd.DataFrame()
tweet_df['cleaned'] = temp['Tweet Content'].apply(preprocessing_text)
tweet_df.head(10)

,cleaned
0,sonshinenewsblast doe_ph nagdeklara ng price f...
1,death toll sa bagyong agatonph lumobo na sa 22...
3,binawi ng national disaster risk reduction man...
6,frontlinetonight plano ng comelec na magpatayo...
11,brgypilar abuyog leyte agatonph abuyogneedshelp
12,death toll sa bagsik ng bagyong agatonph pumal...
13,going na po ang aming livestream cause para po...
14,cash donations channels paypal gcash alab ayro...
23,mga namatay sa hagupit ng agatonph sa visayas ...
24,php 300 nga atuda sa mga taga capiz nga biktim...


In [4]:
sentiment_analyzer = SentimentIntensityAnalyzer()
new_words = pd.read_csv('Datasets/tagalog_lexicon.csv', header=None, index_col=0, squeeze=True).to_dict()
sentiment_analyzer.lexicon.update(new_words)

def polarity(text):
    return sentiment_analyzer.polarity_scores(text)['compound']
tweet_df['sentiment_value'] = tweet_df['cleaned'].apply(polarity)

In [5]:
def sentiment(label):
    if label < 0:
        if label < -0.5:
            return -2
        else:
            return 1
    elif label == 0:
        return 0
    elif label > 0:
        if label > 0.5:
            return 1
        else:
            return 2


In [6]:
tweet_df['sentiment'] = tweet_df['sentiment_value'].apply(sentiment)
tweet_df.sample(10)

,cleaned,sentiment_value,sentiment
16326,ts agaton still eastern visayas ang bagyong ag...,-0.7269,-2
6277,tulong tulong para sa mga kapwa natin pilipino...,0.4767,2
16617,ingat po mga kababayan lalo na sa eastern visa...,-0.5574,-2
8268,kaya natin magpa trend sa mga ganap ng idol na...,0.7003,1
2334,another landslide happening brgy mailhi sitio ...,0.1027,2
8814,iloilo right di pa lahat ng places nkasama d2 ...,0.4019,2
14150,kahuot sa akoa dughan maglantaw sa nahitabo sa...,0.0000,0
8531,translationsunodsunod na ang pagdagsa ng mga p...,-0.3612,1
16212,sanay na sanay na yata ang team namin sa relie...,0.8074,1
13615,look nagsimula nang mag kapinkbisig ang ating ...,0.1531,2


In [7]:
vect = CountVectorizer(ngram_range=(1,2)).fit(tweet_df['cleaned'])
feature_names = vect.get_feature_names()

In [8]:
X = tweet_df['cleaned']
Y = tweet_df['sentiment']
X = vect.transform(X)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [9]:
print("Size of x_train:", (x_train.shape))
print("Size of y_train:", (y_train.shape))
print("Size of x_test:", (x_test.shape))
print("Size of y_test:", (y_test.shape))

Size of x_train: (2712, 42062)
Size of y_train: (2712,)
Size of x_test: (679, 42062)
Size of y_test: (679,)


In [10]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
logreg_pred = logreg.predict(x_test)
logreg_acc = accuracy_score(logreg_pred, y_test)
print("Test accuracy: " + str(logreg_acc*100))

Test accuracy: 71.72312223858616


In [11]:
pred_y_df=pd.DataFrame({'Actual Value':y_test,'Predicted Value':logreg_pred}) 
#pred_y_df =pred_y_df[(pred_y_df["Actual Value"] != pred_y_df["Predicted Value"])]
pred_y_df

,Actual Value,Predicted Value
351,2,1
9889,0,0
13674,2,2
12036,1,-2
7938,1,1
...,...,...
5199,0,0
12296,1,1
16415,0,0
5844,0,0


In [12]:
# Word to predict (single)
word = ["nasalanta bagyo bahay baha patay lunod buhay"]
temp_word = vect.transform(word)
temp_word
logreg.predict(temp_word)

array([-2])

In [13]:
# Real Test
real_df = tweet_df
real_df
temp_word = vect.transform(real_df['cleaned'])
test = logreg.predict(temp_word)
result = pd.DataFrame({'Text':real_df['cleaned'],'Predicted Value':test, 'Actual Value':real_df['sentiment']}) 
result

,Text,Predicted Value,Actual Value
0,sonshinenewsblast doe_ph nagdeklara ng price f...,1,1
1,death toll sa bagyong agatonph lumobo na sa 22...,-2,-2
3,binawi ng national disaster risk reduction man...,-2,-2
6,frontlinetonight plano ng comelec na magpatayo...,-2,-2
11,brgypilar abuyog leyte agatonph abuyogneedshelp,0,0
...,...,...,...
16994,warningmataas na ang tsansa na maging bagyo pr...,-2,-2
16996,tandaan posible pa pong magbago ang forecast n...,-2,-2
16997,kung sakali ang mga susunod na lokal na pangal...,1,1
16998,breaking mataas na ang tiyansang magkaroon ng ...,1,-2


In [14]:
from sklearn.svm import LinearSVC
SVCmodel = LinearSVC()
SVCmodel.fit(x_train, y_train)
svc_pred = SVCmodel.predict(x_test)
svc_acc = accuracy_score(svc_pred, y_test)
print("test accuracy: {:.3f}%".format(svc_acc*100))

test accuracy: 70.103%


In [15]:
from sklearn.ensemble import RandomForestClassifier
naive = RandomForestClassifier()
naive.fit(x_train, y_train)
naive_pred = naive.predict(x_test)
naive_acc = accuracy_score(naive_pred, y_test)
print("test accuracy: {:.3f}%".format(naive_acc*100))

test accuracy: 67.452%


In [16]:
def model_train(obj):
    model_obj = obj
    model_obj.fit(x_train, y_train)
    return model_obj.predict(x_test)

forest_model = RandomForestClassifier()
pred = model_train(forest_model)

forest_acc = accuracy_score(pred, y_test)
print("test accuracy: {:.3f}%".format(forest_acc*100))

test accuracy: 68.041%


In [17]:
ph = pd.read_csv("Datasets/ph.csv")

ph_loc = ph.admin_name
need = ['Biliran', 'Leyte', 'Samar', 'Northern Samar', 'Western Samar', 'Eastern Samar']
ph_loc = ph_loc[ph_loc.isin(need) == True]
ph_loc = ph_loc.reset_index(drop=True)
ph_loc.head()

0             Samar
1             Leyte
2             Samar
3    Northern Samar
4     Eastern Samar
Name: admin_name, dtype: object

In [18]:
# NOT FINAL! RANDOM LOACTION ASSIGMENT
# NOT ACTUAL SPATIAL ANALYSIS
import json
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'

import random

provinces = json.load(open("Datasets/Provinces.geojson", "r"))
id_map = {}
for feature in provinces["features"]:
    feature["id"] = feature["properties"]["ID_1"]
    id_map[feature["properties"]["NAME_1"]] = feature["id"]

rand_loc = pd.DataFrame()

rand = []
for x in range(3391):
    #loc = "Leyte"
    loc = ph_loc[random.randint(0, ph_loc.shape[0] - 1)]
    if not id_map.__contains__(loc):
        loc= "Tarlac"
    rand.append(loc)

location_sentiment_df = pd.DataFrame({'Text':real_df['cleaned'],'Location':rand, 'Sentiment':test}).reset_index(drop=True)
location_sentiment_df

,Text,Location,Sentiment
0,sonshinenewsblast doe_ph nagdeklara ng price f...,Leyte,1
1,death toll sa bagyong agatonph lumobo na sa 22...,Northern Samar,-2
2,binawi ng national disaster risk reduction man...,Samar,-2
3,frontlinetonight plano ng comelec na magpatayo...,Northern Samar,-2
4,brgypilar abuyog leyte agatonph abuyogneedshelp,Leyte,0
...,...,...,...
3386,warningmataas na ang tsansa na maging bagyo pr...,Leyte,-2
3387,tandaan posible pa pong magbago ang forecast n...,Northern Samar,-2
3388,kung sakali ang mga susunod na lokal na pangal...,Leyte,1
3389,breaking mataas na ang tiyansang magkaroon ng ...,Leyte,1


In [19]:
location_sentiment_df = location_sentiment_df[['Location', 'Sentiment']]

In [20]:
location_sentiment_df["id"] = location_sentiment_df["Location"].apply(lambda x: id_map[x])

In [21]:
fig = px.choropleth(
    location_sentiment_df,
    locations="id",
    geojson=provinces,
    color="Sentiment",
    title="PH",
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [22]:
occuring_loc = location_sentiment_df.Location.value_counts().reset_index()['index']
locs_sentiment = pd.DataFrame()
summ_locs_sentiment = pd.DataFrame()
list_mean = []

for _loc in range(occuring_loc.shape[0]): 
    place = occuring_loc[_loc]
    temp = location_sentiment_df.loc[location_sentiment_df["Location"].isin([place])].Sentiment.reset_index(drop=True)
    locs_sentiment[place] = temp
for _loc in range(occuring_loc.shape[0]): 
    place = occuring_loc[_loc]
    list_mean.append(locs_sentiment[place].mean())
    
summ_locs_sentiment['mean'] = pd.Series(list_mean)
summ_locs_sentiment['location'] = occuring_loc
summ_locs_sentiment['id'] = summ_locs_sentiment["location"].apply(lambda x: id_map[x])
summ_locs_sentiment

,mean,location,id
0,0.196817,Leyte,43
1,0.160514,Northern Samar,54
2,0.170635,Samar,66
3,0.140893,Eastern Samar,31
4,0.250000,Biliran,14


In [23]:
fig = px.choropleth_mapbox(
    summ_locs_sentiment,
    locations="id",
    geojson=provinces,
    color="mean",
    title="PH",
    mapbox_style="carto-positron",
    center={"lat": 12, "lon": 122},
    opacity=0.5,
    zoom=5,
)
fig.show()

In [24]:
import folium
from folium.plugins import HeatMap

mapObj = folium.Map([12, 122], zoom_start=6)

data = [
    [12, 122.32345235, 0.1],
    [12, 123.12341425, 0.1],
    [12, 122.9378453, 0.1],
    [12, 122.4932743, 0.1],
    [12, 122.2343, 0.1],
    [12, 122.5322, 0.1],
]

# create heatmap from the data and add to map
HeatMap(data).add_to(mapObj)

mapObj.save("output.html")

In [25]:
# Real Test
real_df = tweet_df
real_df
temp_word = vect.transform(real_df['cleaned'])
test = logreg.predict(temp_word)
result = pd.DataFrame({'Text':real_df['cleaned'],'Predicted Value':test, 'Actual Value':real_df['sentiment']}) 
result

,Text,Predicted Value,Actual Value
0,sonshinenewsblast doe_ph nagdeklara ng price f...,1,1
1,death toll sa bagyong agatonph lumobo na sa 22...,-2,-2
3,binawi ng national disaster risk reduction man...,-2,-2
6,frontlinetonight plano ng comelec na magpatayo...,-2,-2
11,brgypilar abuyog leyte agatonph abuyogneedshelp,0,0
...,...,...,...
16994,warningmataas na ang tsansa na maging bagyo pr...,-2,-2
16996,tandaan posible pa pong magbago ang forecast n...,-2,-2
16997,kung sakali ang mga susunod na lokal na pangal...,1,1
16998,breaking mataas na ang tiyansang magkaroon ng ...,1,-2


In [26]:
from sklearn.svm import LinearSVC
SVCmodel = LinearSVC()
SVCmodel.fit(x_train, y_train)
svc_pred = SVCmodel.predict(x_test)
svc_acc = accuracy_score(svc_pred, y_test)
print("test accuracy: {:.3f}%".format(svc_acc*100))

test accuracy: 70.103%


In [27]:
from sklearn.ensemble import RandomForestClassifier
naive = RandomForestClassifier()
naive.fit(x_train, y_train)
naive_pred = naive.predict(x_test)
naive_acc = accuracy_score(naive_pred, y_test)
print("test accuracy: {:.3f}%".format(naive_acc*100))

test accuracy: 67.305%


In [28]:
def model_train(obj):
    model_obj = obj
    model_obj.fit(x_train, y_train)
    return model_obj.predict(x_test)

forest_model = RandomForestClassifier()
pred = model_train(forest_model)

forest_acc = accuracy_score(pred, y_test)
print("test accuracy: {:.3f}%".format(forest_acc*100))

test accuracy: 68.189%


In [29]:
ph = pd.read_csv("Datasets/ph.csv")

ph_loc = ph.admin_name
#need = ['Biliran', 'Leyte', 'Samar', 'Northern Samar', 'Western Samar', 'Eastern Samar']
#ph_loc = ph_loc[ph_loc.isin(need) == True]
ph_loc = ph_loc[ph_loc.isin(['Leyte']) == True]
ph_loc = ph_loc.reset_index(drop=True)
ph_loc.head()

0    Leyte
1    Leyte
2    Leyte
3    Leyte
4    Leyte
Name: admin_name, dtype: object

In [30]:
# NOT FINAL! RANDOM LOACTION ASSIGMENT
# NOT ACTUAL SPATIAL ANALYSIS
import json
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'

import random

provinces = json.load(open("Datasets/Provinces.geojson", "r"))
id_map = {}
for feature in provinces["features"]:
    feature["id"] = feature["properties"]["ID_1"]
    id_map[feature["properties"]["NAME_1"]] = feature["id"]

rand_loc = pd.DataFrame()

rand = []
for x in range(3391):
    #loc = "Leyte"
    loc = ph_loc[random.randint(0, ph_loc.shape[0] - 1)]
    if not id_map.__contains__(loc):
        loc= "Tarlac"
    rand.append(loc)

location_sentiment_df = pd.DataFrame({'Text':real_df['cleaned'],'Location':rand, 'Sentiment':test}).reset_index(drop=True)
location_sentiment_df

,Text,Location,Sentiment
0,sonshinenewsblast doe_ph nagdeklara ng price f...,Leyte,1
1,death toll sa bagyong agatonph lumobo na sa 22...,Leyte,-2
2,binawi ng national disaster risk reduction man...,Leyte,-2
3,frontlinetonight plano ng comelec na magpatayo...,Leyte,-2
4,brgypilar abuyog leyte agatonph abuyogneedshelp,Leyte,0
...,...,...,...
3386,warningmataas na ang tsansa na maging bagyo pr...,Leyte,-2
3387,tandaan posible pa pong magbago ang forecast n...,Leyte,-2
3388,kung sakali ang mga susunod na lokal na pangal...,Leyte,1
3389,breaking mataas na ang tiyansang magkaroon ng ...,Leyte,1


In [31]:
location_sentiment_df = location_sentiment_df[['Location', 'Sentiment']]
location_sentiment_df
location_sentiment_df["id"] = location_sentiment_df["Location"].apply(lambda x: id_map[x])

In [32]:
fig = px.choropleth(
    location_sentiment_df,
    locations="id",
    geojson=provinces,
    color="Sentiment",
    title="PH",
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [33]:
provinces["features"][42]["geometry"]["coordinates"][2][0]

[[124.669533, 10.485],
 [124.669144, 10.485],
 [124.668648, 10.48505],
 [124.668243, 10.48525],
 [124.667839, 10.48544],
 [124.667587, 10.48578],
 [124.667442, 10.48627],
 [124.667213, 10.48677],
 [124.667, 10.48731],
 [124.667023, 10.48757],
 [124.667053, 10.48787],
 [124.667282, 10.48839],
 [124.667587, 10.48883],
 [124.667976, 10.48916],
 [124.668373, 10.48945],
 [124.668709, 10.48972],
 [124.669037, 10.48998],
 [124.66938, 10.49023],
 [124.669724, 10.49053],
 [124.670097, 10.49078],
 [124.67054, 10.49095],
 [124.670967, 10.49112],
 [124.671341, 10.49133],
 [124.671707, 10.4915],
 [124.67205, 10.49167],
 [124.672394, 10.49183],
 [124.672768, 10.49202],
 [124.673172, 10.49223],
 [124.673599, 10.49249],
 [124.674057, 10.49273],
 [124.674606, 10.49278],
 [124.675163, 10.49278],
 [124.675667, 10.49282],
 [124.676117, 10.49302],
 [124.67662, 10.49306],
 [124.677116, 10.4931],
 [124.677567, 10.49319],
 [124.677994, 10.49283],
 [124.678322, 10.49254],
 [124.678383, 10.49249],
 [124.678688,

In [34]:
provinces["features"][0]["geometry"]["coordinates"][0]
provinces["features"][76]["properties"]

location_sentiment_df
def get_coords(location_id):
    location = provinces["features"][location_id - 1]["geometry"]["coordinates"][20][0]
    #location = location[random.randint(0, len(location) - 1)] random.uniform(10.5, 75.5)
    #long = random.uniform(location[0][0], location[int(len(location)/2)][0])
    #lat = random.uniform(location[int(len(location)/2)][1], location[0][1])
    location = location[random.randint(0,len(location) - 1)]
    #location = [lat, long]
    #return str(location[0]) + "/" + str(location[1])
    return str(location[1]) + "/" + str(location[0])
location_sentiment_df['coordinates'] = location_sentiment_df['id'].apply(get_coords)
location_sentiment_df

,Location,Sentiment,id,coordinates
0,Leyte,1,43,11.21679/125.047943
1,Leyte,-2,43,11.21491/125.048729
2,Leyte,-2,43,11.21769/125.046661
3,Leyte,-2,43,11.21693/125.047081
4,Leyte,0,43,11.21583/125.04715
...,...,...,...,...
3386,Leyte,-2,43,11.21486/125.048477
3387,Leyte,-2,43,11.21672/125.047989
3388,Leyte,1,43,11.21755/125.047287
3389,Leyte,1,43,11.21583/125.04715


In [35]:
heat_list = []

for _loc in range(location_sentiment_df.shape[0]):
    temp = location_sentiment_df.coordinates[_loc].split("/")
    #((-2 * -1) + 2)/4
    heat_list.append([float(temp[0]), float(temp[1]), float(((location_sentiment_df.Sentiment[_loc] * -1) + 2)/4)])
    #heat_list.append([float(temp[0]), float(temp[1]), random.random()])
    pass
heat_list

[[11.21679, 125.047943, 0.25],
 [11.21491, 125.048729, 1.0],
 [11.21769, 125.046661, 1.0],
 [11.21693, 125.047081, 1.0],
 [11.21583, 125.04715, 0.5],
 [11.21693, 125.047081, 1.0],
 [11.21679, 125.047943, 0.25],
 [11.21583, 125.04715, 0.5],
 [11.21659, 125.046661, 0.5],
 [11.21672, 125.047989, 1.0],
 [11.21659, 125.046661, 0.25],
 [11.21486, 125.048477, 0.25],
 [11.21632, 125.048187, 0.25],
 [11.21686, 125.046799, 1.0],
 [11.21672, 125.047989, 1.0],
 [11.21776, 125.046867, 1.0],
 [11.21734, 125.046837, 0.25],
 [11.21507, 125.048058, 1.0],
 [11.21723, 125.046898, 1.0],
 [11.21723, 125.046898, 1.0],
 [11.21493, 125.04882, 0.25],
 [11.21776, 125.046867, 0.25],
 [11.21491, 125.048729, 1.0],
 [11.21583, 125.04715, 1.0],
 [11.21723, 125.046898, 1.0],
 [11.21734, 125.046837, 0.25],
 [11.21659, 125.046661, 0.25],
 [11.21734, 125.046837, 0.5],
 [11.21507, 125.048058, 0.5],
 [11.21659, 125.046661, 0.5],
 [11.21679, 125.047943, 0.25],
 [11.21493, 125.04882, 1.0],
 [11.21679, 125.047943, 1.0],
 [11

In [36]:
import folium
from folium.plugins import HeatMap

mapObj = folium.Map([12, 122], zoom_start=6)

# create heatmap from the data and add to map
HeatMap(heat_list).add_to(mapObj)

mapObj.save("output.html")

In [37]:
occuring_loc = location_sentiment_df.Location.value_counts().reset_index()['index']
locs_sentiment = pd.DataFrame()
summ_locs_sentiment = pd.DataFrame()
list_mean = []

for _loc in range(occuring_loc.shape[0]):
    place = occuring_loc[_loc]
    temp = location_sentiment_df.loc[location_sentiment_df["Location"].isin([place])].Sentiment.reset_index(drop=True)
    locs_sentiment[place] = temp
for _loc in range(occuring_loc.shape[0]):
    place = occuring_loc[_loc]
    list_mean.append(locs_sentiment[place].mean())

summ_locs_sentiment['mean'] = pd.Series(list_mean)
summ_locs_sentiment['location'] = occuring_loc
summ_locs_sentiment['id'] = summ_locs_sentiment["location"].apply(lambda x: id_map[x])
location_sentiment_df

,Location,Sentiment,id,coordinates
0,Leyte,1,43,11.21679/125.047943
1,Leyte,-2,43,11.21491/125.048729
2,Leyte,-2,43,11.21769/125.046661
3,Leyte,-2,43,11.21693/125.047081
4,Leyte,0,43,11.21583/125.04715
...,...,...,...,...
3386,Leyte,-2,43,11.21486/125.048477
3387,Leyte,-2,43,11.21672/125.047989
3388,Leyte,1,43,11.21755/125.047287
3389,Leyte,1,43,11.21583/125.04715


In [38]:
fig = px.choropleth_mapbox(
    summ_locs_sentiment,
    locations="id",
    geojson=provinces,
    color="mean",
    title="PH",
    mapbox_style="carto-positron",
    center={"lat": 12, "lon": 122},
    opacity=0.5,
    zoom=5,
)
fig.show()